In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rcParams
import os
import sys
import json

In [ ]:
### unsolved issues
###  - check the continuity of the trajectory
###  - consider departing lane
###  - consider the side road with no signal
###  - consider turn around
###  - check vehicles directly put into the intersection
###  - check vehicles that do not enter at a port

In [4]:
path_to_data = os.path.join("..", "raw_data", "NGSIM", "sample")
file_name = "sample_trajectory_100000.csv"

path_to_data = os.path.join("..", "raw_data", "NGSIM", "Lankershim-Boulevard-LosAngeles-CA")
file_name = "NGSIM__Lankershim_Vehicle_Trajectories.csv"

path_to_output = os.path.join("..", "data", "NGSIM")
trajectory_file_name = "flow_la.json"
processed_file_name = "processed_trajectory.csv"
evaluation_file_name = "evaluation.txt"

In [11]:
list_inter_id = [1, 2, 3, 4]
list_section_id = [1, 2, 3, 4, 5]
list_main_road_direc = ["NB", "SB"]
list_side_road_direc = ["WB", "EB"]

dic_direction = {
    1: "EB",
    2: "NB",
    3: "WB",
    4: "SB"
}
dic_movement = {
    1: "s",
    2: "l",
    3: "r"
}

# Section_ID_Direcion
dic_main_road_lanes = {
    "main_1_NB": [(1, "s"), (2, "s")],
    "main_1_SB": [(1, "l"), (2, "ls"), (3, "r")],
    "main_2_NB": [(11, "l"), (1, "s"), (2, "s"), (3, "sr"), (4, "r")],
    "main_2_SB": [(1, "s"), (2, "s"), (3, "s")],
    "main_3_NB": [(11, "l"), (1, "s"), (2, "s"), (3, "s"), (31, "sr")],
    "main_3_SB": [(11, "l"), (12, "l"), (1, "s"), (2, "s"), (3, "s"), (31, "r")],
    "main_4_NB": [(11, "l"), (1, "s"), (2, "s"), (3, "s"), (4, "sr")],
    "main_4_SB": [(11, "l"), (1, "s"), (2, "s"), (3, "s"), (4, "r")],
    "main_5_NB": [(1, "s"), (2, "s"), (3, "s"), (4, "s")],
    "main_5_SB": [(11, "l"), (1, "s"), (2, "s"), (3, "s"), (4, "sr")],
}
# Int_ID_Entrance_Direction
dic_side_road_lanes = {
    "side_1_E_WB": [(1, "l"), (2, "r"), (3, "r")],
    "side_2_W_WB": [(1, "s"), (2, "s"), (3, "s")],
    "side_2_W_EB": [(1, "l"), (2, "s"), (3, "r")], # check 
    "side_2_E_WB": [(1, "ls"), (2, "s"), (3, "r")], 
    "side_2_E_EB": [(1, "s"), (2, "s"), (3, "s")],
    "side_23_E_WB": [(1, "r")], # check map
    "side_23_E_EB": [(1, "s")],
    "side_3_W_WB": [(1, "s"), (2, "s")],
    "side_3_W_EB": [(1, "l"), (2, "sr")],
    "side_3_E_WB": [(1, "l"), (2, "sr")], # check
    "side_3_E_EB": [(1, "s"), (2, "s")],
    "side_34_E_WB": [(1, "r")], # check map
    "side_34_E_EB": [(1, "s")],
    "side_4_W_WB": [(1, "s")],
    "side_4_W_EB": [(1, "ls"), (2, "r")],
    "side_4_E_WB": [(1, "l"), (2, "ls"), (3, "r")],
    "side_4_E_EB": [(1, "s"), (2, "s"), (3, "s")],
}

dic_all_road_lanes = {**dic_main_road_lanes, **dic_side_road_lanes}

def reverse_lane_mapping(list_lane):
    dic = {}
    for ind, lane in enumerate(list_lane):
        lane_id, turn = lane
        dic[lane_id] = {"index": ind, "turning": turn}
    return dic
dic_main_road_lanes_mapping = {key: reverse_lane_mapping(lanes) for key, lanes in dic_main_road_lanes.items()}
dic_side_road_lanes_mapping = {key: reverse_lane_mapping(lanes) for key, lanes in dic_side_road_lanes.items()}

In [4]:
dic_all_road_lanes

{'main_1_NB': [(1, 's'), (2, 's')],
 'main_1_SB': [(1, 'l'), (2, 'ls'), (3, 'r')],
 'main_2_NB': [(11, 'l'), (1, 's'), (2, 's'), (3, 'sr'), (4, 'r')],
 'main_2_SB': [(1, 's'), (2, 's'), (3, 's')],
 'main_3_NB': [(11, 'l'), (1, 's'), (2, 's'), (3, 's'), (31, 'sr')],
 'main_3_SB': [(11, 'l'), (12, 'l'), (1, 's'), (2, 's'), (3, 's'), (31, 'r')],
 'main_4_NB': [(11, 'l'), (1, 's'), (2, 's'), (3, 's'), (4, 'sr')],
 'main_4_SB': [(11, 'l'), (1, 's'), (2, 's'), (3, 's'), (4, 'r')],
 'main_5_NB': [(1, 's'), (2, 's'), (3, 's'), (4, 's')],
 'main_5_SB': [(11, 'l'), (1, 's'), (2, 's'), (3, 's'), (4, 'sr')],
 'side_1_E_WB': [(1, 'l'), (2, 'r'), (3, 'r')],
 'side_2_W_WB': [(1, 's'), (2, 's'), (3, 's')],
 'side_2_W_EB': [(1, 'l'), (2, 's'), (3, 'r')],
 'side_2_E_WB': [(1, 'ls'), (2, 's'), (3, 'r')],
 'side_2_E_EB': [(1, 's'), (2, 's'), (3, 's')],
 'side_23_E_WB': [(1, 'r')],
 'side_23_E_EB': [(1, 's')],
 'side_3_W_WB': [(1, 's'), (2, 's')],
 'side_3_W_EB': [(1, 'l'), (2, 'sr')],
 'side_3_E_WB': [(1,

In [5]:
df = pd.read_csv(os.path.join(path_to_data, file_name))

def convert_time(ser):
    time_0 = pd.Timestamp('2005-06-16 15:28:00') # need to confirm with the signal data
    ret = pd.to_datetime(ser, unit="ms") - time_0
    return ret.apply(pd.Timedelta.total_seconds)

df["Direction"] = df["Direction"].apply(lambda r: dic_direction[r])
df["Rela_time"] = convert_time(df["Global_Time"])
df["Sim_time"] = df["Rela_time"].apply(int)

In [8]:
df["Direction"]

0          NB
24         NB
25         NB
26         NB
27         NB
28         NB
29         NB
30         NB
31         NB
32         NB
33         NB
34         NB
35         NB
36         NB
37         NB
38         NB
39         NB
40         NB
41         NB
42         NB
43         NB
44         NB
45         NB
46         NB
47         NB
48         NB
49         NB
50         NB
51         NB
52         NB
           ..
1607072    SB
1607073    SB
1607074    SB
1607075    SB
1607076    SB
1607077    SB
1607078    SB
1607079    SB
1607080    SB
1607081    SB
1607082    SB
1607083    SB
1607084    SB
1607085    SB
1607086    SB
1607087    SB
1607088    SB
1607089    SB
1607090    SB
1607091    SB
1607131    EB
1607132    EB
1607133    EB
1607134    EB
1607135    EB
1607136    EB
1607137    EB
1607138    EB
1607139    EB
1607140    EB
Name: Direction, Length: 1437763, dtype: object

In [7]:
# filter origin
df = df[~df["O_Zone"].isin([104, 106])]
# filter destination
df = df[~df["D_Zone"].isin([204, 206])]
# filter Lane_ID = 0
df = df[~(df["Lane_ID"] == 0)]

In [9]:
def get_side_inter_id(y):
    
    list_side_inter_range = [(10, 100), (330, 490), (1020, 1130), (1470, 1600)]
    cnt = 1
    for low, high in list_side_inter_range:
        if y >= low and y <= high:
#             print(low, high, cnt)
            break
        cnt += 1
    if cnt > len(list_side_inter_range):
        print("incorrect side road")
        sys.exit()
    return cnt
        

# calculate segment and lane assignment
list_seg = []
cnt_ignored_record = 0
for i in range(len(df)):
    
    if i % 50000 == 0:
        print ("{0}/{1}".format(i, len(df)))
        
    r = df.iloc[i]
    if r["Int_ID"] == 0 and r["Section_ID"] != 0:
        # main road or side road without signal
        if r["Direction"] in list_main_road_direc:
            # main road
            seg = "main_{0}_{1}".format(r["Section_ID"], r["Direction"])
        elif r["Direction"] in list_side_road_direc:
            # side road without signal
            if r["Section_ID"] == 3:
                seg = "side_{0}_{1}_{2}".format(23, "E", r["Direction"])
            elif r["Section_ID"] == 4:
                seg = "side_{0}_{1}_{2}".format(34, "E", r["Direction"])
            else:
                ### currently putting these to intersections
                if r["Section_ID"] == 1 and r["O_Zone"] == 102 and r["D_Zone"] == 201:
                    ##### one case is the left turn from 102 to 201
                    seg = "inter_{0}".format(1)
                elif r["Section_ID"] == 2 and r["D_Zone"] == 203 and r["Direction"] == "EB":
                    seg = "inter_{0}".format(2)
                elif r["Section_ID"] == 2 and r["O_Zone"] == 103 and r["Direction"] == "WB":
                    seg = "inter_{0}".format(2)
                elif r["Section_ID"] == 5 and r["O_Zone"] == 108:
                    seg = "inter_{0}".format(4)
                elif r["Section_ID"] == 5 and r["D_Zone"] == 208:
                    seg = "inter_{0}".format(4)
                else:
                    print("not found no_signal side road")
                    print(r)
                    cnt_ignored_record += 1
                    seg = "invalid"
        else:
            print("wrong direction on main road")
            print(r)
            sys.exit()
    elif r["Int_ID"] != 0 and r["Section_ID"] == 0:
        # intersection
        seg = "inter_{0}".format(r["Int_ID"])
    elif r["Int_ID"] == 0 and r["Section_ID"] == 0:
        # side road with signal
        side_inter_id = get_side_inter_id(r["Local_Y"])
        if r["Direction"] in list_side_road_direc:
            if r["Local_X"] > 0: # on the right side of the main road
                seg = "side_{0}_{1}_{2}".format(side_inter_id, "E", r["Direction"])
            elif r["Local_X"] < 0: # on the left side of the main road
                seg = "side_{0}_{1}_{2}".format(side_inter_id, "W", r["Direction"])
            else:
                print("Local_X equals to 0")
                print(r)
                sys.exit()
        else:
            # consider vehicles that are put into the intersection very close to the intersection
            
            seg = "inter_{}".format(side_inter_id)
            ### for instance, 102 -> 207, turning right at 207, it will be north bound
            ### currently putting these to intersections
    else:
        print("both Int_ID and Section_ID not equals to 0")
        sys.exit()
        # error
    list_seg.append(seg)


0/1437763
50000/1437763
100000/1437763
150000/1437763
200000/1437763
250000/1437763
300000/1437763
350000/1437763
400000/1437763
450000/1437763
500000/1437763
550000/1437763
600000/1437763
650000/1437763
700000/1437763
750000/1437763
800000/1437763
850000/1437763
900000/1437763
950000/1437763
1000000/1437763
1050000/1437763
1100000/1437763
1150000/1437763
1200000/1437763
1250000/1437763
1300000/1437763
1350000/1437763
1400000/1437763


In [ ]:
cnt_ignored_record

In [12]:
df["Segment"] = pd.Series(np.array(list_seg), index=df.index)

In [ ]:
# plot the x, y by segment to check

In [13]:
# convert to sim road segment representation
dic_bound_sim_mapping = {"NB": 1, "SB": 3, "WB": 2, "EB": 0}
dic_inter_sim_mapping = {"inter_{}".format(i): "Intersection_1_{0}".format(i) for i in list_inter_id}

def get_main_segment_mapping(segment):
    _, sec, direc = segment.split('_')
    if direc == "NB":
        return "road_{0}_{1}_{2}".format(1, int(sec)-1, dic_bound_sim_mapping[direc])
    elif direc == "SB":
        return "road_{0}_{1}_{2}".format(1, int(sec), dic_bound_sim_mapping[direc])
    else:
        print("wrong direction for main road")
        sys.exit()
dic_main_segment_sim_mapping = {seg: get_main_segment_mapping(seg) for seg in dic_main_road_lanes}

def get_side_segment_mapping(segment):
    _, inter, entrance, direc = segment.split('_')
    if len(inter) == 2:
        # side road with no signal
        return "invalid"
    if entrance == "W" and direc == "WB":
        return "road_{0}_{1}_{2}".format(1, int(inter), dic_bound_sim_mapping[direc])
    elif entrance == "W" and direc == "EB":
        return "road_{0}_{1}_{2}".format(0, int(inter), dic_bound_sim_mapping[direc])
    elif entrance == "E" and direc == "WB":
        return "road_{0}_{1}_{2}".format(2, int(inter), dic_bound_sim_mapping[direc])
    elif entrance == "E" and direc == "EB":
        return "road_{0}_{1}_{2}".format(1, int(inter), dic_bound_sim_mapping[direc])
    else:
        print("wrong direction for side road")
        sys.exit()
dic_side_segment_sim_mapping = {seg: get_side_segment_mapping(seg) for seg in dic_side_road_lanes}
dic_extra_segment_sim_mapping = {'side_1_E_EB': "invalid"}
dic_segment_sim_mapping = {**dic_inter_sim_mapping, **dic_main_segment_sim_mapping, **dic_side_segment_sim_mapping, **dic_extra_segment_sim_mapping}

In [14]:
df["Segment_in_sim"] = df["Segment"].apply(lambda r: dic_segment_sim_mapping[r])
df = df[df["Segment_in_sim"] != "invalid"]

In [15]:
df.to_csv(os.path.join(path_to_output, processed_file_name), index=False)

In [5]:
df = pd.read_csv(os.path.join(path_to_output, processed_file_name))

In [8]:
# convert to input data

def convert_feet_to_meter(l):
    return round(l*0.3048, 2) # set float

def get_vehicle_acc(ser):
    return ser[ser > 0].max()

def get_vehicle_dec(ser):
    return abs(ser[ser < 0].min())

def get_vehicle_min_gap(ser):
    return 2.5/0.3048

def get_vehicle_max_speed(ser):
    return ser.max()

def get_vehicle_headway_time(ser):
    return 1.5

def convert_route(ser):
    list_route = ser.drop_duplicates()
    ret = []
    for s in list_route:
        if s.startswith("Intersection"):
            pass
        else:
            ret.append(s)
    return ret

list_vehicle = []
list_vehicle_id = []
df_gb_vehicle = df.groupby("Vehicle_ID")
list_ports = [
    "road_1_0_1",
    "road_2_1_2",
    "road_2_2_2",
    "road_2_3_2",
    "road_2_4_2",
    "road_1_5_3",
    "road_0_4_0",
    "road_0_3_0",
    "road_0_2_0",
]

cnt = 0

dic_o_zone_to_segment_in_sim = {
    101: "road_1_0_1",
    102: "road_2_1_2",
    103: "road_2_2_2",
    105: "road_2_3_2",
    107: "road_2_4_2",
    108: "road_1_5_3",
    109: "road_0_4_0",
    110: "road_0_3_0",
    111: "road_0_2_0",
}

for vehicle_id, traj in df_gb_vehicle:
    traj = traj.sort_values(by="Global_Time")
    # check whether depart from ports
    if traj["Segment_in_sim"].iloc[0] not in list_ports:
        if "Intersection" in traj["Segment_in_sim"].iloc[0]:
            # may assign intersection cars to its origin
            traj["Segment_in_sim"].iloc[0] = dic_o_zone_to_segment_in_sim[traj["O_Zone"].iloc[0]]
        else:       
            cnt += 1
            continue
        
    dic_vehicle = {
        "vehicle": {
          "length": traj.iloc[0]["v_length"],
          "width": traj.iloc[0]["v_Width"],
          "maxPosAcc": convert_feet_to_meter(get_vehicle_acc(traj["v_Acc"])),
          "maxNegAcc": convert_feet_to_meter(get_vehicle_dec(traj["v_Acc"])),
          "usualPosAcc": convert_feet_to_meter(get_vehicle_acc(traj["v_Acc"])),
          "usualNegAcc": convert_feet_to_meter(get_vehicle_dec(traj["v_Acc"])),
          "minGap": convert_feet_to_meter(get_vehicle_min_gap(traj["Space_Headway"])),
          "maxSpeed": convert_feet_to_meter(get_vehicle_max_speed(traj["v_Vel"])),
          "headwayTime": get_vehicle_headway_time(traj["Time_Headway"])
        },
        "route": convert_route(traj["Segment_in_sim"]),
        "interval": 1,
        "startTime": int(traj.iloc[0]["Rela_time"]),
        "endTime": int(traj.iloc[0]["Rela_time"])
    }
    
    ### may need to check the continuity of the trajectory
    if len(dic_vehicle["route"]) < 2:
        cnt += 1
        continue
    list_vehicle.append(dic_vehicle)
    list_vehicle_id.append(traj.iloc[0]["Vehicle_ID"])
json.dump(list_vehicle, open(os.path.join(path_to_output, trajectory_file_name), "w"), indent=2)

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
df = df[df["Vehicle_ID"].isin(list_vehicle_id)]

In [10]:
# calculate Rela_X and Rela_Y
# if it is on main road, copy Local_Y and convert Local_X
# if it is on side road, copy Local_X and convert Local_Y
lane_width_in_feet = 3 / 0.3048

def convert_main_road_lane_to_coordinate(seg, l_id, l_width, direc):
    
    try:
        l_ind = dic_main_road_lanes_mapping[seg][l_id]["index"]
        if direc == "NB":
            return (l_ind + 0.5) * l_width
        elif direc == "SB":
            return - (l_ind + 0.5) * l_width
        else:
            print("should have removed side roads without signal")
            sys.exit()
    except KeyError:
        return float("nan")

def convert_side_road_lane_to_coordinate(seg, l_id, l_width, direc):
    
    dic_side_road_local_y_offset = {
        "side_1": 10,
        "side_2": 410,
        "side_3": 1075,
        "side_4": 1535,
    }
    
    try:
        l_ind = dic_side_road_lanes_mapping[seg][l_id]["index"]
        memo = '_'.join(seg.split('_')[:2])
        if memo in dic_side_road_local_y_offset:
            offset = dic_side_road_local_y_offset[memo]
            if "W_WB" in seg:
                return (l_ind + 0.5) * l_width + offset
            elif "W_EB" in seg:
                return - (l_ind + 0.5) * l_width + offset
            elif "E_WB" in seg:
                return (l_ind + 0.5) * l_width + offset
            elif "E_EB" in seg:
                return - (l_ind + 0.5) * l_width + offset
            else:
                print("side road lane calculation error")
                sys.exit()
        else:
            # side road without signal
            print("should have removed side roads without signal")
            sys.exit()
    except KeyError:
        return float("nan")
        
def convert_lane_to_coordinate(r):
    seg, lane_id, direction = r[["Segment", "Lane_ID", "Direction"]]
    if seg.startswith("main"):
        y = r["Local_Y"]
        x = convert_main_road_lane_to_coordinate(seg, lane_id, lane_width_in_feet, direction)
    elif seg.startswith("side"):
        x = r["Local_X"]
        y = convert_side_road_lane_to_coordinate(seg, lane_id, lane_width_in_feet, direction)
    elif seg.startswith("inter"):
        x = r["Local_X"]
        y = r["Local_Y"]
    else:
        print("convert lane error")
        sys.exit()
    return x, y

array_rela_xy = df.apply(lambda r: convert_lane_to_coordinate(r), axis=1)
df[["Rela_X", "Rela_Y"]] = array_rela_xy.apply(pd.Series)

NameError: ("name 'dic_main_road_lanes_mapping' is not defined", 'occurred at index 1')

In [ ]:
df.dropna(inplace=True)
# map the coordinate to sim coordinate
x_0 = 0
y_0 = 10
df["Sim_X"] = (df["Rela_X"] - x_0) * 0.3048
df["Sim_Y"] = (df["Rela_Y"] - y_0) * 0.3048

In [ ]:
# getting the local Y and averaging is not accurate
# def get_side_road_offset():
#     dic_single_direction = {}
#     for key in dic_side_road_lanes:
#         if "W_WB" in key:
#             offset = df[df["Segment"] == key]["Local_Y"].min()
#         elif "W_EB" in key:
#             offset = df[df["Segment"] == key]["Local_Y"].max()
#         elif "E_WB" in key:
#             offset = df[df["Segment"] == key]["Local_Y"].min()
#         elif "E_EB" in key:
#             offset = df[df["Segment"] == key]["Local_Y"].max()
#         dic_single_direction[key] = offset
#     dic_double_direction = {}
#     for key, offset in dic_single_direction.items():
#         print(key[:-3])
#         if key[:-3] not in dic_double_direction:
#             dic_double_direction[key[:-3]] = []
#         dic_double_direction[key[:-3]].append(offset)
#     print(dic_double_direction)
#     dic_ret = {}
#     for key, l in dic_double_direction.items():
#         dic_ret[key] = np.average(l)
#     return dic_ret
# dic_side_road_local_y_offset = get_side_road_offset()

In [22]:
dic_segment_to_segment_in_sim = dict(df[["Segment", "Segment_in_sim"]].drop_duplicates().sort_values(by="Segment_in_sim").values)
dic_all_road_lanes_in_sim = {}
for segment, segment_in_sim in dic_segment_to_segment_in_sim.items():
    if not segment.startswith("inter"):
        dic_all_road_lanes_in_sim[segment_in_sim] = dic_all_road_lanes[segment]
list_all_lanes = []
for segment_in_sim, list_lanes in dic_all_road_lanes_in_sim.items():
    list_all_lanes += [segment_in_sim+'_'+str(i) for i in range(len(list_lanes))]

In [24]:
json.dump(dic_all_road_lanes_in_sim, open(os.path.join(path_to_output, "lane_setting.json"), "w"))

In [20]:
# convert to evaluation data
# def convert_to_evaluation_data(df)
f_evaluation = open(os.path.join(path_to_output, evaluation_file_name), "w")
df_gb_sim_time = df.groupby(by="Sim_time", sort=True)
for t, t_group in df_gb_sim_time:
    if t  == 0:
        continue
    ret = "{0:.1f}".format(t)
    ret += ",lane_speed "
    list_lane_speed = []
    for lane in list_all_lanes:
        road = lane[:-2]
        l_id = int(lane[-1])
        list_vec = t_group[(t_group["Segment_in_sim"] == road) & (t_group["Lane_ID"] == l_id)]
        list_lane_speed.append(
            lane + ':' + ("{:.4f}".format(list_vec["v_Vel"].apply(convert_feet_to_meter).mean()) if len(list_vec) else "0"))
    ret += " ".join(list_lane_speed)
    f_evaluation.write(ret+'\n')
f_evaluation.close()

NameError: name 'df_gb_sim_time' is not defined

In [ ]:
df

In [ ]:
df_gb_sim_time

In [ ]:
# check time
np.all((df["Rela_time"] > 0).values)

In [ ]:
df_sub = df[(df["Int_ID"] == 0) & (df["Section_ID"] == 0)]
# df_sub[df_sub["O_Zone"].isin([102, 103, 104, 105, 106, 107, 109, 110, 111]) 
#        | df_sub["D_Zone"].isin([203, 204, 205, 206, 207, 209, 210, 211])]
df_sub

In [ ]:
# check vel, acc, dec, length, space headway
df.hist(figsize=(32, 18))

In [ ]:
fig, ax = plt.subplots()
counts, bins, patche = ax.hist(df_sub["Local_Y"], bins=list(range(0, 1680, 10)))

In [ ]:
np.where(counts != 0)

In [ ]:
df.columns

In [ ]:
df